In [330]:
import random
import time
import math

In [399]:
class Solucion:
    def __init__(self, solucion, domain = [(0,9)] * 12):
        self.solucion = solucion
        self.domain = domain

    def neighbors(self):
        '''
            regresa una lista con las soluciones que se forman al mover un vuelo uno arriba o uno abajo
        '''
        neighbors = []
        s = self.solucion
        for i in range(len(self.domain)):
            
            if s[i] > self.domain[i][0]:
                neighbors.append(Solucion(s[0:i] + [s[i] - 1] + s[i+1:]))
            if s[i] < self.domain[i][1]:
                neighbors.append(Solucion(s[0:i] + [s[i] + 1] + s[i+1:]))
        return neighbors

    def __str__(self):
        return str(self.solucion)

    def __repr__(self):
        return self.__str__()

    def __iter__(self):
        return self.solucion

    def __len__(self):
        return len(self.solucion)
        
    def __getitem__(self, indice):
        return self.solucion[indice]



In [536]:
class Flights:
    def __init__(self, path_flights, people, destination):
        self.destination = destination
        self.people = people
        self.flights = {}
        
        with open(path) as f:
            for line in f:
                origin, dest, t_depart, t_arrive, price = line.strip().split(',')
                self.flights.setdefault((origin, dest), [])
                self.flights[(origin, dest)].append((
                    self.get_minutes(t_depart), 
                    self.get_minutes(t_arrive), 
                    int(price)
                ))
    def print_sol(self, s):
        assert isinstance(s, Solucion), 's debe ser un objeto tipo Solucion'
        for i in range(len(s)//2):
            
            dep = self.get_dep(i)
            persona = self.people[i][0]
            
            hora_dep1 = self.get_flights_dep(persona)[s[i*2]][0]
            hora_ar1 = self.get_flights_dep(persona)[s[i*2]][1]
            costo_dep1 = self.get_flights_dep(persona)[s[i*2]][2]
            
            hora_dep1 = self.get_hours(hora_dep1)
            hora_ar1 = self.get_hours(hora_ar1)
            costo_dep1 = '$' + str(costo_dep1).ljust(7, ' ')

            hora_dep2 = self.get_flights_ret(persona)[s[i*2]][0]
            hora_ar2 = self.get_flights_ret(persona)[s[i*2]][1]
            costo_dep2 = self.get_flights_ret(persona)[s[i*2]][2]
            
            hora_dep2 = self.get_hours(hora_dep2)
            hora_ar2 = self.get_hours(hora_ar2)
            costo_dep2 = '$' + str(costo_dep2).ljust(7, ' ')
            
        
            
            print(dep, 
                  persona.ljust(10, ' '), 
                  hora_dep1,
                  hora_ar1,
                  costo_dep1,
                  hora_dep2,
                  hora_ar2,
                  costo_dep2)
            
    
    def get_dep(self, nombre):
        if isinstance(nombre, str):
            for tupla in self.people:
                if tupla[0] == nombre:
                    return tupla[1]
        if isinstance(nombre, int):
            return self.people[nombre][1]
        

    def get_flights_dep(self, nombre):
        #asserts
        dep = self.get_dep(nombre)
        
        return self.flights[(dep,self.destination)]
            
    def get_flights_ret(self, nombre):
        #asserts
        dep = self.get_dep(nombre)
        return self.flights[(self.destination, dep)]
            
    def random_solucion(self):
        solucion = []
        for tupla in self.people:
            nombre = tupla[0]
            solucion.append(
                random.choice(range(len(self.get_flights_dep(nombre))))
            )
            solucion.append(
                random.choice(range(len(self.get_flights_ret(nombre))))
            )
        return Solucion(solucion)

    def get_minutes(self, t):
        x = time.strptime(t, '%H:%M')
        h = x.tm_hour
        m = x.tm_min
        return 60 * h + m
    
    def get_hours(self, t):
        h = t // 60
        m = t - h * 60 
        h = str(h).rjust(2, '0')       
        m = str(m).rjust(2, '0')
        return h + ':' + m
        
        
    def schedule_cost(self, solucion):
        # contamos el precio total de cada vuelo (ida y regreso)
        total_price = 0
        s = solucion.solucion
        # nos interesa conocer el tiempo de llegada a NY mas tarde
        # y el tiempo de salida de NY mas temprano.
        latest_arrival = 0
        earliest_departure = 24 * 60
        
        for i in range(len(s) // 2):
            origin = self.people[i][1]
            out_flight = self.flights[(origin, self.destination)][s[2*i]]
            ret_flight = self.flights[(self.destination, origin)][s[2*i+1]]
            
            total_price += out_flight[2] # vuelo de ida
            total_price += ret_flight[2] # vuelo de regreso
            
            # tiempo de llegada máximo
            # tiempo de salida mínimo
            if latest_arrival < out_flight[1]:
                latest_arrival = out_flight[1]
            if earliest_departure > ret_flight[0]:
                earliest_departure = ret_flight[0]
        
        # contamos el tiempo de espera de cada persona
        total_wait = 0
        
        for i in range(len(s) // 2):
            origin = self.people[i][1]
            out_flight = self.flights[(origin, self.destination)][s[2*i]]
            ret_flight = self.flights[(self.destination, origin)][s[2*i+1]]
            
            # todos esperan al último familiar en llegar
            total_wait += latest_arrival - out_flight[1]
            
            # todos llegan al aeropuerto al mismo tiempo y esperan su vuelo
            total_wait += ret_flight[0] - earliest_departure
            
            # si el último en llegar a NY llega después del primero en
            # irse de NY se paga un día más de la renta del carro.
            # el costo de la renta por un día es independiente de la
            # solución.
            if latest_arrival > earliest_departure:
                total_price += 50
        
        # El costo total es el precio total de los vuelos y el tiempo de
        # espera total de las personas.
        # Buscamos soluciones con un bajo costo.
        return total_price + total_wait
    




        
    

In [537]:
class Optimizador:
    def __init__(self, fun_cost, flights):
        self.fun_cost = fun_cost
        self.flights = flights
        
    def solve_randomly(self, repeats = 1000):
        '''
            Devuelvela mejor solucion encontrada de manera aleatorio
        '''
        best_cost = float('inf')
        best_sol = None
        
        for _ in range(repeats):
            s = self.flights.random_solucion()
            c = self.fun_cost(s)
            if c < best_cost:
                best_cost = c
                best_sol = s
        
        return best_sol

    def solve_hillclimbing(self, repeats = 1000):
        s = self.flights.random_solucion()
        contador = 0
        while (True and contador <= 1000):
            contador = contador + 1
            neighbors = s.neighbors()
            cost = self.fun_cost(s)
            best_neighbor = min(neighbors, key=self.fun_cost)
            neighbor_cost = self.fun_cost(best_neighbor)
            
            if cost <= neighbor_cost:
                return s
            
            s = best_neighbor
    def solve_annealing(self, Ti=10000.0, Tf=0.1, alpha=0.95):
        cost_of = self.fun_cost
        solution = self.flights.random_solucion()
        cost = cost_of(solution)
        T = Ti
        while T > Tf:
            neighbor = random.choice(solution.neighbors())
            neighbor_cost = cost_of(neighbor)
            diff = cost - neighbor_cost
            if diff > 0 or random.random() < math.exp(diff / T):
                solution = neighbor
                cost = neighbor_cost
            T = alpha*T
        
        return solution
        


    

In [538]:
path = 'assets/schedule.txt'
people = [('Seymour', 'BOS'),
          ('Franny', 'DAL'),
          ('Zooey', 'CAK'),
          ('Walt', 'MIA'),
          ('Buddy', 'ORD'),
          ('Les', 'OMA')]
destination = 'LGA'

In [539]:
yo = Flights(path, people, destination)

In [540]:
yo.schedule_cost(Solucion([0,9]*6))

3348

In [541]:
optimizador = Optimizador(yo.schedule_cost, yo)

In [542]:
optimizador.solve_annealing()

[7, 3, 5, 5, 6, 3, 6, 9, 6, 3, 6, 3]

In [543]:
s = yo.random_solucion()

In [544]:
type(s)

__main__.Solucion

In [545]:
yo.print_sol(s)

BOS Seymour    09:45 11:50 $172     09:58 11:18 $130    
DAL Franny     16:52 20:48 $448     17:14 20:59 $277    
CAK Zooey      17:08 19:08 $262     16:33 18:15 $253    
MIA Walt       19:53 22:21 $173     20:27 23:42 $169    
ORD Buddy      09:42 11:32 $169     09:11 10:42 $172    
OMA Les        06:11 08:31 $249     06:19 08:13 $239    


In [525]:
yo.get_hours(128)

'02:08'